## 3. Domain Propagation方法

本notebook对应[这一篇blog post](https://hanqiu92.github.io/blogs/2020/IP_BB_solver_3_202007/)中的内容，主要包括B&B方法中domain propagation步骤中线性约束的处理方法实现。

首先是准备工作：导入相关的计算工具包、从文件util_lec_3.py中读取上一个notebook中完成的内容。

In [1]:
import time
import math
import numpy as np
from cylp.cy import CyClpSimplex
from util import *
from util_lec_3 import *
np.set_printoptions(suppress=True,precision=4)

下面的代码包括以下两部分：

* 对原有**Node类**和**BBSolver类**的扩展
* 在**Propagate类**中实现基于线性约束的变量界加强方法(bound tightening)，并增加相应的类型标签

Node类的扩展主要是在activate方法中调用propagate方法加强变量上下界。

In [2]:
class NodeExtend(Node):
    def activate(self,solver):
        super().activate(solver)
        ## 调用propagator处理变量上下界，并判断子问题可行性
        self.infeas_flag,self.local_vars_lb_dict,self.local_vars_ub_dict = \
            solver.propagator.propagate(solver,self.local_vars_lb_dict,self.local_vars_ub_dict,self.var_idx)

BBSolver类的扩展则主要包括对Propagator的调用以及对提前发现的不可行结点的处理。

In [3]:
class BBSolverExtend(BBSolver):        
    def reset_propagate_type(self,propagate_type):
        ## 设置propagate方法
        self.propagator = Propagator(self.problem,propagate_type)
        
    def __init__(self,problem,branch_type=None,select_type=None,propagate_type=None):
        ## 进行基础的初始化
        super().__init__(problem,branch_type,select_type)
        ## 增加对propagate方法的选取
        self.reset_propagate_type(propagate_type)
        
    def node_process(self,node_idx):
        '''
        加入propagate后的不可行性处理
        '''
        if node_idx not in self.unprocessed_node_idxs:
            print(self.unprocessed_node_idxs,node_idx)
            
        node = self.nodes[node_idx]
        self.unprocessed_node_idxs.remove(node_idx)

        if node.LB >= self.global_obj:
            ## 可以直接剪枝
            node.mark_processed()
            prun_status = 0
            return prun_status

        node.activate(self) ## 激活node，生成局部信息
        self.modify_problem_bounds(node) ## 更新问题接口中的变量上下界
        
        ## 增加对可行性的判断
        if not node.infeas_flag:
            ## 原有逻辑
            self.node_solve(node) ## 求解子问题
        else:
            ## 新增逻辑：直接更新目标值和下界估计
            node.LB = node.obj = MAX

        ## 更新BB树的下界信息
        node_child = node
        update_lb_flag = True
        while (node_child.level > 0) and update_lb_flag:
            node_parent = node_child.parent
            curr_LB = MAX
            for child in node_parent.childs:
                if child.LB < curr_LB:
                    curr_LB = child.LB
                if not child.is_processed:
                    ## no need to further update LB
                    update_lb_flag = False
                    break
            node_parent.LB = curr_LB
            node_child = node_parent  
        self.global_LB = self.root_node.LB

        prun_status = 0
        if not node.sol_feas and node.LB < self.global_obj:
            ## 子问题整数不可行且子问题最优解小于当前最优整数解，继续branch生成子问题
            self.branch(node)
            prun_status = 1

        self.recover_problem_bounds(node) ## 恢复问题接口中的变量上下界
            
        node.deactivate() ## 释放内存
        return prun_status
        
    def create_node(self,parent_node,var_idx,var_lb,var_ub,var_sol,var_bnd):
        '''
        相比原来版本，只是将Node改为Node Extend
        '''
        new_node = NodeExtend(node_idx=self.next_node_idx,parent=parent_node,cost_manager=self.cost_manager,
                        var_idx=var_idx,var_lb=var_lb,var_ub=var_ub,var_sol=var_sol,var_bnd=var_bnd,
                        init_var_lb=self.init_vars_lb[var_idx],init_var_ub=self.init_vars_ub[var_idx])
        parent_node.childs += [new_node]
        self.add_node(new_node)
        
    def solve_root_node(self):
        '''
        相比原来版本，只是将Node改为Node Extend
        '''
        ## 生成根结点
        root_node = NodeExtend()
        self.root_node = root_node
        self.add_node(root_node)
        ## 执行简化版的process
        self.unprocessed_node_idxs.remove(root_node.idx)
        root_node.activate(self) ## 激活node，生成局部信息
        self.node_solve(root_node) ## 求解子问题
        self.root_sol = root_node.sol ## 根结点（初始问题）的LP relaxation最优解
        self.global_LB = root_node.LB ## 更新BB树的上下界信息
        status = SolveStatus.ONGOING
        if not root_node.sol_feas and self.global_LB < self.global_obj:
            self.branch(root_node)
        else:
            status = SolveStatus.OPT
        root_node.deactivate() ## 释放内存
        return status

在完成了上述准备后，下面，我们开始实现Propagator。在这个notebook中，我们主要实现对线性约束的处理：每次调用时，先根据变量上下界计算约束的隐含上下界，然后再根据约束的上下界和隐含上下界推断其他变量的上下界；如果得到了更强的上下界，则进行更新，然后重复前述流程。

在实现过程中，需要特别注意以下几个细节：

* 计算效率：在propgate的过程中，存在较多if-else判断逻辑，如果用numpy做矢量化则计算效率较低；这里，我们实现最原始的elementwise计算逻辑，然后通过numba进行jit加速。
* 数值误差处理：变量和约束的上下界可能为无穷，这在计算中通常用一个较大的数表示；这导致如果该值与其他正常数值混合在一起进行计算，则容易出现数值误差。为此，我们引入了变量inf和val来分开记录无穷值和有限值。
* 内存管理/数据结构：在此前实现的B&B主流程中，我们通过dict管理子问题的变量上下界信息，以节省内存及便于理解；但是，dict的相关计算效率低，不适用于propagate这种计算密集任务。因此，这里引入了get_vars_bnd_array和get_vars_bnd_dict两个函数来将dict转化为(sparse) array。（注：在B&B整个流程中，可以通过numba+array with duplicates的方式获取更好的计算效率，但是需要引入一个cache来对上下界变动信息进行统一的内存管理，否则容易发生内存泄漏。）

具体的实现逻辑可以参考下面代码中的注释。

In [4]:
@unique
class PropagateType(Enum):
    ## 子问题上下界处理的方法类型
    NONE = 1 ## 不做处理
    LINEAR = 2 ## 按线性约束处理
    LINEAR_OBJ = 3 ## 按线性约束处理，加上对目标函数的处理

class Propagator:
    def __init__(self,problem,process_type=PropagateType.NONE):
        ## 确定propagate方法类型
        self.propagate_type = propagate_type
        self.propagate = self.propagate_linear_obj
        if self.propagate_type == PropagateType.NONE:
            self.propagate = self.propagate_none
        elif self.propagate_type == PropagateType.LINEAR:
            self.propagate = self.propagate_linear
        elif self.propagate_type == PropagateType.LINEAR_OBJ:
            self.propagate = self.propagate_linear_obj

        ## 初始化辅助变量
        ### 提前提取问题的线性约束信息，节省后续计算开销
        bl,bu = problem.constraintsLower.copy(),problem.constraintsUpper.copy()
        self.row_bnds = self.bound_init(bl,bu)
        A_csr = problem.coefMatrix.tocsr()
        self.As = self.matrix_init(A_csr)
        ### 提前提取问题的线性约束信息+目标函数信息，节省后续计算开销
        bl,bu = problem.constraintsLower.copy(),problem.constraintsUpper.copy()
        bl_extend,bu_extend = np.concatenate([[-MAX],bl]),np.concatenate([[MAX],bu])
        self.row_bnds_extend = self.bound_init(bl_extend,bu_extend)
        A_csr_extend = sp.vstack([csr_matrix(problem.objective.copy()),A_csr])
        self.As_extend = self.matrix_init(A_csr_extend)

    def propagate_none(self,solver,local_vars_lb_dict,local_vars_ub_dict,update_var_idx):
        ## 直接返回原始上下界信息
        return False,local_vars_lb_dict,local_vars_ub_dict

    def propagate_linear(self,solver,local_vars_lb_dict,local_vars_ub_dict,update_var_idx):
        ## 处理线性约束
        infeas_flag,local_vars_lb_dict,local_vars_ub_dict = \
            self.bound_tighten(self.As,solver.int_vars_bool,update_var_idx,
                               self.row_bnds,solver.vars_lb.copy(),solver.vars_ub.copy(),
                               local_vars_lb_dict,local_vars_ub_dict)
        return infeas_flag,local_vars_lb_dict,local_vars_ub_dict

    def propagate_linear_obj(self,solver,local_vars_lb_dict,local_vars_ub_dict,update_var_idx):
        ## 处理线性约束 + 目标函数
        if solver.global_obj < INF:
            self.row_bnds_extend[1][0] = solver.global_obj
            self.row_bnds_extend[3][0] = 0
            infeas_flag,local_vars_lb_dict,local_vars_ub_dict = \
                self.bound_tighten(self.As_extend,solver.int_vars_bool,update_var_idx,
                                   self.row_bnds_extend,solver.vars_lb.copy(),solver.vars_ub.copy(),
                                   local_vars_lb_dict,local_vars_ub_dict)
        else:
            return self.propagate_linear(solver,local_vars_lb_dict,local_vars_ub_dict,update_var_idx)
        return infeas_flag,local_vars_lb_dict,local_vars_ub_dict

    def bound_init(self,l,u):
        '''
        上下界处理：将上下界展开成有限值+无限值的形式
        '''
        l_inf,u_inf = l <= -INF,u >= INF
        l_val,u_val = l,u
        l_val[l_inf] = 0
        u_val[u_inf] = 0
        bnds = (l_val,u_val,l_inf.astype(np.float64),u_inf.astype(np.float64))
        return bnds
    
    def matrix_init(self,A_csr):
        '''
        约束系数矩阵处理：提前提取出正和负值元素
        '''
        A_posi,A_nega = A_csr.multiply(A_csr > 0),A_csr.multiply(A_csr < 0)
        A_csr_nnz,A_posi_nnz,A_nega_nnz = A_csr.copy(),A_posi.copy(),A_nega.copy()
        A_csr_nnz.data = np.ones((len(A_csr_nnz.data),),dtype=A_csr_nnz.data.dtype)
        A_posi_nnz.data = np.ones((len(A_posi_nnz.data),),dtype=A_posi_nnz.data.dtype)
        A_nega_nnz.data = np.ones((len(A_nega_nnz.data),),dtype=A_nega_nnz.data.dtype)
        As = (A_csr.tocsc(),A_csr,A_csr_nnz,A_posi,A_nega,A_posi_nnz,A_nega_nnz)
        return As
    
    def bound_tighten(self,As,is_int_bools,update_var_idx,row_bnds,l_init,u_init,l_new_dict,u_new_dict):
        '''
        加强上下界的主流程
        '''
        ## 变量上下界预处理：将dict转为array
        var_bnds,var_bnd_chgs = self.get_vars_bnd_array(l_init,u_init,l_new_dict,u_new_dict)
        ## 计算约束隐含上下界
        row_im_bnds = self.calc_cons_im_bound(As,var_bnds)
        ## 初始化未处理行标签
        A_csc = As[0]
        n,m = A_csc.shape
        row_unprocess_flag = np.full((n,),False,dtype=np.bool)
        row_redundant_flag = np.full((n,),False,dtype=np.bool)
        idxs = range(A_csc.indptr[update_var_idx],A_csc.indptr[update_var_idx+1])
        rows = A_csc.indices[idxs]
        row_unprocess_flag[rows] = True
        iter_ = 0
        ## 检查剩余未处理行
        infeas_flag,unprocess_rows = self.check_row_status(row_unprocess_flag,row_redundant_flag,
                                                     row_bnds,row_im_bnds)
        while(len(unprocess_rows) > 0) and (not infeas_flag) and (iter_ < 10):
            iter_ += 1
            ## 行处理：根据约束上下界推断变量上下界
            var_bnds,var_bnd_chgs,var_bnds_delta,col_unprocess_flag = \
                self.update_vars_bound(As,is_int_bools,var_bnds,var_bnd_chgs,
                                       row_bnds,row_im_bnds,unprocess_rows)
            ## 列处理：根据变量上下界变动更新约束的隐含上下界
            if np.any(col_unprocess_flag):
                row_im_bnds,row_unprocess_flag = \
                    self.update_cons_im_bound(As,var_bnds_delta,row_im_bnds,
                                              col_unprocess_flag,row_unprocess_flag)
            ## 检查剩余未处理行
            infeas_flag,unprocess_rows = self.check_row_status(row_unprocess_flag,row_redundant_flag,
                                                     row_bnds,row_im_bnds)
            
        ## 变量上下界后处理：将array转成dict
        l_new_dict,u_new_dict = self.get_vars_bnd_dict(var_bnds,var_bnd_chgs,l_init,u_init,
                                                       l_new_dict,u_new_dict)
        return infeas_flag,l_new_dict,u_new_dict
    
    def get_vars_bnd_array(self,l_init,u_init,l_new_dict,u_new_dict):
        '''
        变量上下界预处理：将dict转为array
        '''
        l,u = l_init.copy(),u_init.copy()
        for (var_idx,init_var_bnd),var_bnd in l_new_dict.items():
            l[var_idx] = var_bnd
        for (var_idx,init_var_bnd),var_bnd in u_new_dict.items():
            u[var_idx] = var_bnd
        var_bnds = self.bound_init(l,u)
        l_chgs,u_chgs = np.full((len(l),),False),np.full((len(u),),False)
        return var_bnds,(l_chgs,u_chgs)
    
    def get_vars_bnd_dict(self,var_bnds,var_bnd_chgs,l_init,u_init,l_new_dict,u_new_dict):
        '''
        变量上下界后处理：将array转成dict
        '''
        l_val,u_val,l_inf,u_inf = var_bnds
        l_chg,u_chg = var_bnd_chgs
        for var_idx in np.where(l_chg)[0]:
            l_new_dict[(var_idx,l_init[var_idx])] = l_val[var_idx]
        for var_idx in np.where(u_chg)[0]:
            u_new_dict[(var_idx,u_init[var_idx])] = u_val[var_idx]
        return l_new_dict,u_new_dict
    
    def calc_cons_im_bound(self,As,var_bnds):
        '''
        计算约束隐含上下界，包括有限值和无限值两部分
        '''
        A_csc,A_csr,A_csr_nnz,A_posi,A_nega,A_posi_nnz,A_nega_nnz = As
        l_val,u_val,l_inf,u_inf = var_bnds
        bl_im_val = A_posi._mul_vector(l_val) + A_nega._mul_vector(u_val)
        bu_im_val = A_posi._mul_vector(u_val) + A_nega._mul_vector(l_val)
        bl_im_inf = A_posi_nnz._mul_vector(l_inf) + A_nega_nnz._mul_vector(u_inf)
        bu_im_inf = A_posi_nnz._mul_vector(u_inf) + A_nega_nnz._mul_vector(l_inf)
        row_im_bnds = (bl_im_val,bu_im_val,bl_im_inf,bu_im_inf)
        return row_im_bnds

    def update_cons_im_bound(self,As,var_bnds_delta,row_im_bnds,col_unprocess_flag,row_unprocess_flag):
        '''
        列处理：根据变量上下界变动更新约束的隐含上下界
        '''
        cols = np.where(col_unprocess_flag)[0]
        bl_im_val,bu_im_val,bl_im_inf,bu_im_inf = row_im_bnds
        if len(cols) < 0.1 * len(col_unprocess_flag):
            ## 稀疏更新：通过numba实现jit加速
            A_csc = As[0]
            l_val_delta,u_val_delta,l_inf_delta,u_inf_delta = var_bnds_delta
            update_cons_im_bnd_jit(cols,row_unprocess_flag,
                                    A_csc.indptr,A_csc.indices,A_csc.data,
                                    bl_im_val,bu_im_val,bl_im_inf,bu_im_inf,
                                    l_val_delta,u_val_delta,l_inf_delta,u_inf_delta)
        else:
            ## 批量更新：调用calc_cons_im_bound直接计算隐含上下界的变动量
            A_csr_nnz = As[2]
            row_unprocess_flag = A_csr_nnz._mul_vector(col_unprocess_flag) > 0
            bl_im_val_delta,bu_im_val_delta,bl_im_inf_delta,bu_im_inf_delta = \
                self.calc_cons_im_bound(As,var_bnds_delta)
            bl_im_val,bu_im_val = bl_im_val+bl_im_val_delta,bu_im_val+bu_im_val_delta
            bl_im_inf,bu_im_inf = bl_im_inf+bl_im_inf_delta,bu_im_inf+bu_im_inf_delta
        row_im_bnds = (bl_im_val,bu_im_val,bl_im_inf,bu_im_inf)
        return row_im_bnds,row_unprocess_flag
        
    def update_vars_bound(self,As,is_int_bools,var_bnds,var_bnd_chgs,row_bnds,row_im_bnds,unprocess_rows):
        '''
        行处理：根据约束上下界推断变量上下界
        '''
        l_val,u_val,l_inf,u_inf = var_bnds
        l_chg,u_chg = var_bnd_chgs
        bl_val,bu_val,bl_inf,bu_inf = row_bnds
        bl_im_val,bu_im_val,bl_im_inf,bu_im_inf = row_im_bnds
        ## 根据约束隐含上下界获取变量的隐含上下界：通过numba实现jit加速
        num_var = len(l_val)
        l_new,u_new = np.full((num_var,),-INF),np.full((num_var,),INF)
        A_csr = As[1]
        get_new_var_bnd_jit(unprocess_rows,A_csr.indptr,A_csr.indices,A_csr.data,
                           l_val,u_val,l_inf,u_inf,
                           bl_val,bu_val,bl_inf,bu_inf,
                           bl_im_val,bu_im_val,bl_im_inf,bu_im_inf,
                           l_new,u_new)
        ## 判断变量的隐含上下界是否可以加强原始上下界：通过numba实现jit加速
        l_val_delta,u_val_delta = np.zeros((num_var,),dtype=np.float64),np.zeros((num_var,),dtype=np.float64)
        l_inf_delta,u_inf_delta = np.zeros((num_var,),dtype=np.float64),np.zeros((num_var,),dtype=np.float64)
        col_unprocess_flag = np.full((num_var,),False,dtype=np.bool)
        process_new_var_bnd_jit(l_val,u_val,l_inf,u_inf,l_chg,u_chg,is_int_bools,l_new,u_new,
                               l_val_delta,u_val_delta,l_inf_delta,u_inf_delta,col_unprocess_flag)
        var_bnds = (l_val,u_val,l_inf,u_inf)
        var_bnd_chgs = (l_chg,u_chg)
        var_bnds_delta = (l_val_delta,u_val_delta,l_inf_delta,u_inf_delta)
        return var_bnds,var_bnd_chgs,var_bnds_delta,col_unprocess_flag
        
    def check_row_status(self,row_unprocess_flag,row_redundant_flag,row_bnds,row_im_bnds):
        '''
        检查剩余未处理行
        '''
        bl_val,bu_val,bl_inf,bu_inf = row_bnds
        bl_im_val,bu_im_val,bl_im_inf,bu_im_inf = row_im_bnds
        ## 通过numba实现jit加速
        return check_row_status_jit(row_unprocess_flag,row_redundant_flag,
                                   bl_val,bu_val,bl_inf,bu_inf,
                                   bl_im_val,bu_im_val,bl_im_inf,bu_im_inf)

    
#############################
## 下面是numba实现jit加速的部分
import numba as nb
from numba import njit,types

@njit
def check_row_status_jit(row_unprocess_flag,row_redundant_flag,
                       bl_val,bu_val,bl_inf,bu_inf,
                       bl_im_val,bu_im_val,bl_im_inf,bu_im_inf):
    '''
    检查未处理行
    '''
    num_rows = len(row_unprocess_flag)
    infeas_flag = False
    rows = list()
    for row in range(num_rows):
        if row_unprocess_flag[row] and not row_redundant_flag[row]:
            redundant_flag_lower = redundant_flag_upper = False
            row_unprocess_flag[row] = False
            ## 判断不可行性
            if bu_im_inf[row] == 0 and bl_inf[row] == 0 and bl_val[row] > bu_im_val[row] + TOL:
                infeas_flag = True
                break
            if bl_im_inf[row] == 0 and bu_inf[row] == 0 and bu_val[row] < bl_im_val[row] - TOL:
                infeas_flag = True
                break
            ## 判断是否多余（给定约束被隐含约束覆盖）
            if (bl_im_val[row] >= bl_val[row] - TOL and bl_im_inf[row] == 0) or (bl_inf[row] > 0):
                redundant_flag_lower = True
            if (bu_im_val[row] <= bu_val[row] + TOL and bu_im_inf[row] == 0) or (bu_inf[row] > 0):
                redundant_flag_upper = True
            if redundant_flag_lower and redundant_flag_upper:
                row_redundant_flag[row] = True
            else:
                rows.append(row)
    rows = np.array(rows,dtype=np.int32)
    return infeas_flag,rows
    
@njit
def get_new_var_bnd_jit(rows,indptr,indices,data,
                       l_val,u_val,l_inf,u_inf,
                       bl_val,bu_val,bl_inf,bu_inf,
                       bl_im_val,bu_im_val,bl_im_inf,bu_im_inf,
                       l_new,u_new):
    '''
    对每个行，计算implied变量上下界，然后进行聚合。implied上下界的通用计算形式：
    对约束系数为正的情况，变量隐含上(下)界 = (约束实际上(下)界 - 约束隐含下(上)界) / 约束系数 + 变量实际下(上)界
    '''
    for row in rows:
        for idx in range(indptr[row],indptr[row+1]):
            col,aij = indices[idx],data[idx]
            
            ## 首先判断计算公式内各项是否有限值；若否，则不需要进一步计算
            if aij > 0:
                bl_im_inf_ij = bl_im_inf[row] - l_inf[col]
                bu_im_inf_ij = bu_im_inf[row] - u_inf[col]
            else:
                bl_im_inf_ij = bl_im_inf[row] - u_inf[col]
                bu_im_inf_ij = bu_im_inf[row] - l_inf[col]
            
            if bu_inf[row] == 0 and bl_im_inf_ij == 0:
                ## bl_im和bu都是有限值，界估计结果可用
                delta = (bu_val[row] - bl_im_val[row]) / aij
                if aij > 0:
                    delta += l_val[col]
                    u_new[col] = min(u_new[col],delta)
                else:
                    delta += u_val[col]
                    l_new[col] = max(l_new[col],delta)
                
            if bl_inf[row] == 0 and bu_im_inf_ij == 0:
                ## bu_im和bl都是有限值，界估计结果可用
                delta = (bl_val[row] - bu_im_val[row]) / aij
                if aij > 0:
                    delta += u_val[col]
                    l_new[col] = max(l_new[col],delta)
                else:
                    delta += l_val[col]
                    u_new[col] = min(u_new[col],delta)

@njit
def process_new_var_bnd_jit(l_val,u_val,l_inf,u_inf,l_chg,u_chg,is_int_bools,l_new,u_new,
                               l_val_delta,u_val_delta,l_inf_delta,u_inf_delta,col_unprocess_flag):
    '''
    对每个变量，处理隐含上下界估计，判断是否可以加强原始上下界
    '''
    num_var = len(l_new)    
    for idx in range(num_var):
        ## 处理下界估计
        l_new_val = l_new[idx]
        if l_new_val > -INF:
            update_flag = False
            l_new_val = 1e-5 * math.floor(1e5 * l_new_val + TOL) ## 数值误差处理
            ## 整数rounding
            if is_int_bools[idx]:
                l_new_val = math.ceil(l_new_val - TOL)
            if l_inf[idx] > 0:
                update_flag = True ## 从无限值 -> 有限值
            else:
                ## 判断界的优化效果是否超过一定阈值
                thres = abs(l_val[idx])
                if u_inf[idx] == 0:
                    thres = min(u_val[idx]-l_val[idx],thres)
                thres = 0.05 * max(thres,1)
                if l_new_val > l_val[idx] + thres:
                    update_flag = True
            ## 更新变量下界
            if update_flag:
                col_unprocess_flag[idx] = True
                l_val_delta[idx],l_inf_delta[idx] = l_new_val-l_val[idx],-l_inf[idx]
                l_chg[idx],l_val[idx],l_inf[idx] = True,l_new_val,0
             
        ## 处理上界估计
        u_new_val = u_new[idx]
        if u_new_val < INF:
            update_flag = False
            u_new_val = 1e-5 * math.ceil(1e5 * u_new_val - TOL) ## 数值误差处理
            ## 整数rounding
            if is_int_bools[idx]:
                u_new_val = math.floor(u_new_val + TOL)
            if u_inf[idx] > 0:
                update_flag = True ## 从无限值 -> 有限值
            else:
                ## 判断界的优化效果是否超过一定阈值
                thres = abs(u_val[idx])
                if l_inf[idx] == 0:
                    thres = min(u_val[idx]-l_val[idx],thres)
                thres = 0.05 * max(thres,1)
                if u_new_val < u_val[idx] - thres:
                    update_flag = True
            ## 更新变量上界
            if update_flag:
                col_unprocess_flag[idx] = True
                u_val_delta[idx],u_inf_delta[idx] = u_new_val-u_val[idx],-u_inf[idx]
                u_chg[idx],u_val[idx],u_inf[idx] = True,u_new_val,0

@njit
def update_cons_im_bnd_jit(cols,row_unprocess_flag,indptr,indices,data,
                            bl_im_val,bu_im_val,bl_im_inf,bu_im_inf,
                            l_val_delta,u_val_delta,l_inf_delta,u_inf_delta):
    '''
    对每个更新列，稀疏更新相应的隐含上下界估计
    '''
    for col in cols:
        for idx in range(indptr[col],indptr[col+1]):
            row,aij = indices[idx],data[idx]
            row_unprocess_flag[row] = True
            if aij > 0:
                bl_im_val[row] += aij * l_val_delta[col]
                bu_im_val[row] += aij * u_val_delta[col]
                bl_im_inf[row] += l_inf_delta[col]
                bu_im_inf[row] += u_inf_delta[col]
            else:
                bl_im_val[row] += aij * u_val_delta[col]
                bu_im_val[row] += aij * l_val_delta[col]
                bl_im_inf[row] += u_inf_delta[col]
                bu_im_inf[row] += l_inf_delta[col]

下面，我们沿用已有的求解入口来对上述propagate方法进行测试。

In [5]:
from util import *

def bb_solve(model_name,branch_type,select_type,propagate_type,time_limit=3600,if_disp=True):
    ## 读取问题
    s = CyClpSimplex()
    s.readMps(model2fname(model_name))
    s.logLevel = 0
    solver = BBSolverExtend(s,branch_type,select_type,propagate_type)
    dt,status,LB,obj = solver.solve(time_limit=time_limit,if_disp=if_disp,disp_iters=100000)
    total_node_cnt,remain_node_cnt = len(solver.nodes),len(solver.unprocessed_node_idxs)
    for node in solver.nodes:
        node.clear()
    return dt,status,LB,obj,total_node_cnt,remain_node_cnt,solver

## 读取测试案例
with open('data/test_cases_v3','r+') as f:
    lines = f.readlines()
model_names = [line.strip('\n') for line in lines]

In [6]:
#############################
branch_type = BranchType.RELIABLE
select_type = SelectType.BE_PLUNGE
propagate_type = PropagateType.LINEAR_OBJ
#############################

max_secs = 600 ## 每个问题限时10分钟
result = dict()
for model_name in model_names:
    dt,status,LB,obj,total_node_cnt,remain_node_cnt,_ = \
        bb_solve(model_name,branch_type,select_type,propagate_type,time_limit=max_secs,if_disp=False)
    print('{}: time={:.2f}, status={}. LB={:.3e}, obj={:.3e}. total nodes={}, remain nodes={}.'.format(
        model_name,dt,status,LB,obj,total_node_cnt,remain_node_cnt))
    result[model_name] = (dt,status,LB,obj,total_node_cnt,remain_node_cnt)

## 保存测试结果
## 对于不同的branching/node selection/propagate配置，记得修改保存的文件名，以免覆盖原来的结果
df_result = process_result_new(result,save_fname='result/ours_benchmark_v3_rel_bep.csv')
print('elapsed hours: {:.2f}.'.format(df_result['time'].sum() / 3600))

binkar10_1: time=600.00, status=SolveStatus.ONGOING. LB=6.663e+03, obj=6.747e+03. total nodes=400281, remain nodes=133835.
dano3_3: time=98.30, status=SolveStatus.OPT. LB=5.763e+02, obj=5.763e+02. total nodes=109, remain nodes=0.
dano3_5: time=601.26, status=SolveStatus.ONGOING. LB=5.762e+02, obj=5.769e+02. total nodes=867, remain nodes=218.
eil33-2: time=276.31, status=SolveStatus.OPT. LB=9.340e+02, obj=9.340e+02. total nodes=21397, remain nodes=0.
gen-ip002: time=375.45, status=SolveStatus.ONGOING. LB=-4.829e+03, obj=-4.776e+03. total nodes=1334699, remain nodes=334698.
gen-ip054: time=383.83, status=SolveStatus.ONGOING. LB=6.773e+03, obj=6.853e+03. total nodes=1342945, remain nodes=342944.
gmu-35-40: time=600.00, status=SolveStatus.ONGOING. LB=-2.407e+06, obj=1.000e+20. total nodes=903573, remain nodes=236264.
gmu-35-50: time=600.00, status=SolveStatus.ONGOING. LB=-2.608e+06, obj=1.000e+20. total nodes=749829, remain nodes=288164.
icir97_tension: time=600.00, status=SolveStatus.ONGO

通过上述测试，我们可以得到以下结论：

* 引入propagate会明显降低计算效率（同样时间限制内遍历更少结点）。
* 引入propagate后似乎可以找到更多可行解，但是效果不稳定：如果用pseudo cost branching + best estimate w/ plunging，可以在两个gmu-35测试问题中找到可行解；如果用reliable branching + best estimate w/ plunging，则可以在测试问题icir97_tension中找到可行解。这三个问题在之前的测试中是找不到可行解的。
* 在遍历同样结点数时，引入propagate对下界估计有一定优化效果，但对大部分测试问题几乎没有区别。